In [1]:
import time
import argparse
import numpy as np

import torch
import torch.optim as optim

import torch.nn as nn

from utils import load_data,normalize,toy_data,nmi_score,modularity_matrix,modularity
from models import GNN

import community as community_louvain
from networkx import from_numpy_matrix
import networkx as nx

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

from sklearn.cluster import SpectralClustering

import pycombo

import pandas as pd

torch.set_printoptions(sci_mode=False)

from utils import doublerelu


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
cuda = torch.cuda.is_available()
weight_decay = 10e-4
epochs = 1000
seed = 17

In [4]:
class GNN1Layer(Module):

    def __init__(self, batch_size, in_features, out_features):
        super(GNN1Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size

        weight1_eye = torch.FloatTensor(torch.eye(in_features, out_features))
        weight1_eye = weight1_eye.reshape((1, in_features, out_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        self.weight1 = Parameter(weight1_eye)
        self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))

    def forward(self, input, adj):
        v1 = torch.bmm(input, self.weight1)
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        output = v1 + v2
        return output

In [5]:
class GNN2Layer(Module):

    def __init__(self, batch_size, in_features, out_features):
        super(GNN2Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size
        weight1_eye = torch.FloatTensor(torch.eye(in_features, in_features))
        weight1_eye = weight1_eye.reshape((1, in_features, in_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        weight1_rand = torch.empty(batch_size,in_features,out_features-in_features)
        torch.nn.init.xavier_uniform_(weight1_rand, gain=1.0)
        self.weight1 = Parameter(torch.cat((weight1_eye,weight1_rand),dim=2))
        self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))

    def forward(self, input, adj):
        v1 = torch.bmm(input, self.weight1)
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        output = v1 + v2
        return output

In [6]:
class GNN1(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden):
        super(GNN1, self).__init__()

        self.gc1 = GNN1Layer(batch_size, ndim, ndim)

    def forward(self, x, adj):
        x = doublerelu(self.gc1(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        return x

In [7]:
class GNN2(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden):
        super(GNN2, self).__init__()

        self.gc1 = GNN2Layer(batch_size, ndim, hidden)
        self.gc2 = GNN1Layer(batch_size, hidden, ndim)

    def forward(self, x, adj):
        x = doublerelu(self.gc1(x, adj))
        x = doublerelu(self.gc2(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        return x

In [8]:
def community_detection(G,method,GNN,features=None,hidden=1,seed=seed):
    
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed(seed)
    
    adj = nx.to_numpy_matrix(G)
    adj = np.expand_dims(adj, axis=0)
        
    #Combo
    combo_partition = pycombo.execute(G)
    nb_comm = max(combo_partition[0].values())+1
    combo_mod = combo_partition[1]

    adj_norm = normalize(adj)

    adj = torch.FloatTensor(np.array(adj))
    adj_norm = torch.FloatTensor(np.array(adj_norm))
    
        
    if type(features) == type(None):

        # features
        if method == "louvain":
            partition = community_louvain.best_partition(G)
            #get binary matrix of the partition
            nb_community = max(list(partition.values())) + 1
            communities =  np.array(list(partition.values())).reshape(-1)
            C = np.eye(nb_community)[communities]
            C = torch.FloatTensor(C)
            #perturbations
            C = perturb(C)
        elif method == "spectral":
            partition = spectralPartition(adj,nb_comm)
            C = partition.float()
            nb_community = C.shape[1]

        features = torch.FloatTensor(C)
        features = features.unsqueeze(0)
    
    if method == "louvain":
        lr = 0.001
    elif method == "spectral":
        lr = 0.001
        
    nb_community = features.shape[-1]
    
    Q = modularity_matrix(adj)
    
    # Model and optimizer

    model = GNN(batch_size=adj.shape[0],
                nfeat=adj.shape[1],
                ndim=nb_community,
                hidden=hidden)

    if cuda:
        model.cuda()
        features = features.cuda()
        adj = adj.cuda()
        adj_norm = adj_norm.cuda()
        Q = Q.cuda()

    # Train model
    t_total = time.time()

    optimizer = optim.Adam(model.parameters(),
                           lr=lr, weight_decay=weight_decay)
    

    for epoch in range(epochs):

        t = time.time()
        model.train()
        optimizer.zero_grad()

        C = model(features, adj_norm)
        loss = modularity(C,Q)

        loss.backward(retain_graph=True)

        optimizer.step()

        if epoch == 0:
            best_loss = loss
            init_mod = loss
            best_C = C
        else:
            if loss < best_loss:
                best_loss = loss
                best_C = C

        if epoch % 100 == 0:
            print('Epoch: {:04d}'.format(epoch + 1),
                  'Modularity: {:.8f}'.format(-best_loss.item()),
                  'time: {:.4f}s'.format(time.time() - t))

    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    
    return nb_community,-init_mod,-best_loss,combo_mod, best_C

In [9]:
def perturb(A):
    p = 0.4
    epsilon = torch.rand(A.shape).uniform_(0, p) - torch.rand(A.shape).uniform_(0, p)
    return torch.clip(A + epsilon,0,1)

In [10]:
def spectralPartition(A,nb_comm):
    A = A[0] # TODO 3D
    clustering = SpectralClustering(n_clusters=nb_comm,assign_labels='discretize',random_state=0).fit(A)
    clusters = list(clustering.labels_)
    partitions = torch.tensor(clusters)
    return torch.nn.functional.one_hot(partitions)

In [11]:
def exp1(hidden):
    G = [nx.karate_club_graph(),nx.les_miserables_graph(),nx.florentine_families_graph()]
    methods = ["louvain","spectral"]
    finetuned_mods = []
    networks = ["Karate Club","Karate Club","Les Miserables","Les Miserables","Florentine","Florentine"]
    for network in G:
        for method in methods:
            nb_community,init_mod,finetuned_mod,combo_mod,features = community_detection(network,method,GNN1,hidden=hidden)
            finetuned_mods.append(finetuned_mod.cpu().detach().numpy()[0][0])
            
    return finetuned_mods

In [12]:
def exp2(hidden):
    G = [nx.karate_club_graph(),nx.les_miserables_graph(),nx.florentine_families_graph()]
    methods = ["louvain","spectral"]
    nb_communities = []
    init_mods = []
    finetuned_mods = []
    combo_mods = []
    methodss = []
    networks = ["Karate Club","Karate Club","Les Miserables","Les Miserables","Florentine","Florentine"]
    hidden_dim = []
    one_layer_mod = exp1(hidden)
    for network in G:
        for method in methods:
            flag = True
            for i in range(10):
                seed = np.random.randint(10000)
                nb_community,init_mod,finetuned_mod,combo_mod,features = community_detection(network,method,GNN1,hidden=hidden,seed=seed)
                initial_mod = torch.clone(init_mod)
                nb_community,init_mod,finetuned_mod,combo_mod,features = community_detection(network,method,GNN2,features=features,hidden=nb_community+hidden,seed=seed)
                if flag:
                    nb_communities.append(nb_community)
                    init_mods.append(initial_mod.cpu().detach().numpy()[0][0])
                    finetuned_mods.append(finetuned_mod.cpu().detach().numpy()[0][0])
                    combo_mods.append(combo_mod)
                    methodss.append(method)
                    hidden_dim.append(nb_community+hidden)
                    flag = False
                else:
                    if finetuned_mod.cpu().detach().numpy()[0][0] > finetuned_mods[-1]:
                        finetuned_mods[-1] = finetuned_mod.cpu().detach().numpy()[0][0]
            
            
    df = pd.DataFrame([networks,methodss,nb_communities,hidden_dim,init_mods,one_layer_mod,finetuned_mods,combo_mods]).transpose()
    df.columns = ["Network","Method","Communities","Hidden Dimensionality","Initial Modularity","1 Layer Finetuned Moduclarity","2 Layer Finetuned Moduclarity","Combo Modularity"] 
    return df

In [13]:
dfs = exp2(1)
for hidden in range(2,11):
    dfs = pd.concat([dfs,exp2(hidden)])

Epoch: 0001 Modularity: 0.25446442 time: 0.3840s
Epoch: 0101 Modularity: 0.38794592 time: 0.0010s
Epoch: 0201 Modularity: 0.41832262 time: 0.0010s
Epoch: 0301 Modularity: 0.41978961 time: 0.0020s
Epoch: 0401 Modularity: 0.41978961 time: 0.0020s
Epoch: 0501 Modularity: 0.41978961 time: 0.0020s
Epoch: 0601 Modularity: 0.41978961 time: 0.0020s
Epoch: 0701 Modularity: 0.41978961 time: 0.0020s
Epoch: 0801 Modularity: 0.41978961 time: 0.0020s
Epoch: 0901 Modularity: 0.41978961 time: 0.0020s
Optimization Finished!
Total time elapsed: 2.0855s
Epoch: 0001 Modularity: 0.34015450 time: 0.0020s
Epoch: 0101 Modularity: 0.35263938 time: 0.0010s
Epoch: 0201 Modularity: 0.35894844 time: 0.0010s
Epoch: 0301 Modularity: 0.36297059 time: 0.0020s
Epoch: 0401 Modularity: 0.36976776 time: 0.0020s
Epoch: 0501 Modularity: 0.39869636 time: 0.0010s
Epoch: 0601 Modularity: 0.39907956 time: 0.0020s
Epoch: 0701 Modularity: 0.39907956 time: 0.0020s
Epoch: 0801 Modularity: 0.39907956 time: 0.0010s
Epoch: 0901 Modula

Epoch: 0201 Modularity: 0.39201188 time: 0.0020s
Epoch: 0301 Modularity: 0.39201188 time: 0.0030s
Epoch: 0401 Modularity: 0.39201188 time: 0.0020s
Epoch: 0501 Modularity: 0.39201188 time: 0.0030s
Epoch: 0601 Modularity: 0.39201188 time: 0.0020s
Epoch: 0701 Modularity: 0.39201188 time: 0.0030s
Epoch: 0801 Modularity: 0.39201188 time: 0.0020s
Epoch: 0901 Modularity: 0.39201188 time: 0.0030s
Optimization Finished!
Total time elapsed: 2.7222s
Epoch: 0001 Modularity: 0.23771280 time: 0.0020s
Epoch: 0101 Modularity: 0.39578980 time: 0.0020s
Epoch: 0201 Modularity: 0.41747940 time: 0.0010s
Epoch: 0301 Modularity: 0.41958153 time: 0.0020s
Epoch: 0401 Modularity: 0.41967103 time: 0.0010s
Epoch: 0501 Modularity: 0.41969803 time: 0.0010s
Epoch: 0601 Modularity: 0.41972500 time: 0.0020s
Epoch: 0701 Modularity: 0.41975009 time: 0.0010s
Epoch: 0801 Modularity: 0.41977125 time: 0.0020s
Epoch: 0901 Modularity: 0.41978693 time: 0.0010s
Optimization Finished!
Total time elapsed: 1.3486s
Epoch: 0001 Modu

Epoch: 0401 Modularity: 0.36976776 time: 0.0010s
Epoch: 0501 Modularity: 0.39869636 time: 0.0010s
Epoch: 0601 Modularity: 0.39907956 time: 0.0020s
Epoch: 0701 Modularity: 0.39907956 time: 0.0010s
Epoch: 0801 Modularity: 0.39907956 time: 0.0010s
Epoch: 0901 Modularity: 0.39907956 time: 0.0010s
Optimization Finished!
Total time elapsed: 1.2128s
Epoch: 0001 Modularity: 0.39907956 time: 0.0020s
Epoch: 0101 Modularity: 0.39907956 time: 0.0020s
Epoch: 0201 Modularity: 0.39907956 time: 0.0020s
Epoch: 0301 Modularity: 0.39907956 time: 0.0020s
Epoch: 0401 Modularity: 0.39907956 time: 0.0020s
Epoch: 0501 Modularity: 0.39907956 time: 0.0020s
Epoch: 0601 Modularity: 0.39907956 time: 0.0030s
Epoch: 0701 Modularity: 0.39907956 time: 0.0020s
Epoch: 0801 Modularity: 0.39907956 time: 0.0030s
Epoch: 0901 Modularity: 0.39907956 time: 0.0030s
Optimization Finished!
Total time elapsed: 2.3733s
Epoch: 0001 Modularity: 0.34015450 time: 0.0010s
Epoch: 0101 Modularity: 0.35263938 time: 0.0010s
Epoch: 0201 Modu

Epoch: 0601 Modularity: 0.39907956 time: 0.0020s
Epoch: 0701 Modularity: 0.39907956 time: 0.0040s
Epoch: 0801 Modularity: 0.39907956 time: 0.0020s
Epoch: 0901 Modularity: 0.39907956 time: 0.0020s
Optimization Finished!
Total time elapsed: 2.3548s
Epoch: 0001 Modularity: 0.26651746 time: 0.0020s
Epoch: 0101 Modularity: 0.53986025 time: 0.0020s
Epoch: 0201 Modularity: 0.56128401 time: 0.0020s
Epoch: 0301 Modularity: 0.56494546 time: 0.0020s
Epoch: 0401 Modularity: 0.56646228 time: 0.0010s
Epoch: 0501 Modularity: 0.56658375 time: 0.0010s
Epoch: 0601 Modularity: 0.56662738 time: 0.0010s
Epoch: 0701 Modularity: 0.56665868 time: 0.0010s
Epoch: 0801 Modularity: 0.56667876 time: 0.0010s
Epoch: 0901 Modularity: 0.56668794 time: 0.0010s
Optimization Finished!
Total time elapsed: 1.2467s
Epoch: 0001 Modularity: 0.56668794 time: 0.0030s
Epoch: 0101 Modularity: 0.56668794 time: 0.0020s
Epoch: 0201 Modularity: 0.56668794 time: 0.0020s
Epoch: 0301 Modularity: 0.56668794 time: 0.0030s
Epoch: 0401 Modu

Epoch: 0801 Modularity: 0.56629837 time: 0.0010s
Epoch: 0901 Modularity: 0.56629837 time: 0.0010s
Optimization Finished!
Total time elapsed: 1.5744s
Epoch: 0001 Modularity: 0.56629837 time: 0.0030s
Epoch: 0101 Modularity: 0.56629837 time: 0.0020s
Epoch: 0201 Modularity: 0.56629837 time: 0.0030s
Epoch: 0301 Modularity: 0.56629837 time: 0.0030s
Epoch: 0401 Modularity: 0.56629837 time: 0.0030s
Epoch: 0501 Modularity: 0.56629837 time: 0.0030s
Epoch: 0601 Modularity: 0.56629837 time: 0.0020s
Epoch: 0701 Modularity: 0.56629837 time: 0.0030s
Epoch: 0801 Modularity: 0.56629837 time: 0.0020s
Epoch: 0901 Modularity: 0.56629837 time: 0.0030s
Optimization Finished!
Total time elapsed: 2.8675s
Epoch: 0001 Modularity: 0.28180939 time: 0.0020s
Epoch: 0101 Modularity: 0.54103386 time: 0.0020s
Epoch: 0201 Modularity: 0.56236607 time: 0.0010s
Epoch: 0301 Modularity: 0.56496549 time: 0.0010s
Epoch: 0401 Modularity: 0.56591797 time: 0.0020s
Epoch: 0501 Modularity: 0.56619143 time: 0.0020s
Epoch: 0601 Modu

...

Epoch: 0601 Modularity: 0.56541568 time: 0.0040s
Epoch: 0701 Modularity: 0.56541568 time: 0.0030s
Epoch: 0801 Modularity: 0.56541568 time: 0.0030s
Epoch: 0901 Modularity: 0.56541568 time: 0.0030s
Optimization Finished!
Total time elapsed: 3.3436s
Epoch: 0001 Modularity: 0.25387222 time: 0.0020s
Epoch: 0101 Modularity: 0.53074467 time: 0.0030s
Epoch: 0201 Modularity: 0.56213462 time: 0.0020s
Epoch: 0301 Modularity: 0.56416810 time: 0.0020s
Epoch: 0401 Modularity: 0.56462741 time: 0.0020s
Epoch: 0501 Modularity: 0.56498241 time: 0.0020s
Epoch: 0601 Modularity: 0.56525886 time: 0.0020s
Epoch: 0701 Modularity: 0.56541300 time: 0.0020s
Epoch: 0801 Modularity: 0.56541568 time: 0.0020s
Epoch: 0901 Modularity: 0.56541568 time: 0.0020s
Optimization Finished!
Total time elapsed: 1.6936s
Epoch: 0001 Modularity: 0.56541568 time: 0.0040s
Epoch: 0101 Modularity: 0.56541568 time: 0.0030s
Epoch: 0201 Modularity: 0.56541568 time: 0.0030s
Epoch: 0301 Modularity: 0.56541568 time: 0.0020s
Epoch: 0401 Modu

Epoch: 0801 Modularity: 0.39308828 time: 0.0010s
Epoch: 0901 Modularity: 0.39308828 time: 0.0020s
Optimization Finished!
Total time elapsed: 1.6087s
Epoch: 0001 Modularity: 0.39308828 time: 0.0030s
Epoch: 0101 Modularity: 0.42710942 time: 0.0030s
Epoch: 0201 Modularity: 0.46980852 time: 0.0020s
Epoch: 0301 Modularity: 0.46980852 time: 0.0020s
Epoch: 0401 Modularity: 0.46980852 time: 0.0030s
Epoch: 0501 Modularity: 0.46980852 time: 0.0030s
Epoch: 0601 Modularity: 0.46980852 time: 0.0030s
Epoch: 0701 Modularity: 0.46980852 time: 0.0030s
Epoch: 0801 Modularity: 0.46980852 time: 0.0030s
Epoch: 0901 Modularity: 0.46980852 time: 0.0020s
Optimization Finished!
Total time elapsed: 2.6388s
Epoch: 0001 Modularity: 0.11745612 time: 0.0010s
Epoch: 0101 Modularity: 0.14251940 time: 0.0020s
Epoch: 0201 Modularity: 0.17311493 time: 0.0020s
Epoch: 0301 Modularity: 0.23827371 time: 0.0020s
Epoch: 0401 Modularity: 0.39308828 time: 0.0020s
Epoch: 0501 Modularity: 0.39308828 time: 0.0010s
Epoch: 0601 Modu

Optimization Finished!
Total time elapsed: 2.4922s
Epoch: 0001 Modularity: 0.11745612 time: 0.0020s
Epoch: 0101 Modularity: 0.14251940 time: 0.0020s
Epoch: 0201 Modularity: 0.17311493 time: 0.0020s
Epoch: 0301 Modularity: 0.23827371 time: 0.0020s
Epoch: 0401 Modularity: 0.39308828 time: 0.0020s
Epoch: 0501 Modularity: 0.39308828 time: 0.0010s
Epoch: 0601 Modularity: 0.39308828 time: 0.0010s
Epoch: 0701 Modularity: 0.39308828 time: 0.0020s
Epoch: 0801 Modularity: 0.39308828 time: 0.0010s
Epoch: 0901 Modularity: 0.39308828 time: 0.0010s
Optimization Finished!
Total time elapsed: 1.7142s
Epoch: 0001 Modularity: 0.39308828 time: 0.0040s
Epoch: 0101 Modularity: 0.42660689 time: 0.0040s
Epoch: 0201 Modularity: 0.47073013 time: 0.0030s
Epoch: 0301 Modularity: 0.47419024 time: 0.0030s
Epoch: 0401 Modularity: 0.47419024 time: 0.0040s
Epoch: 0501 Modularity: 0.47419024 time: 0.0030s
Epoch: 0601 Modularity: 0.47419024 time: 0.0030s
Epoch: 0701 Modularity: 0.47419024 time: 0.0040s
Epoch: 0801 Modu

Epoch: 0101 Modularity: 0.39750004 time: 0.0030s
Epoch: 0201 Modularity: 0.39750004 time: 0.0030s
Epoch: 0301 Modularity: 0.39750004 time: 0.0020s
Epoch: 0401 Modularity: 0.39750004 time: 0.0030s
Epoch: 0501 Modularity: 0.39750004 time: 0.0020s
Epoch: 0601 Modularity: 0.39750004 time: 0.0030s
Epoch: 0701 Modularity: 0.39750004 time: 0.0030s
Epoch: 0801 Modularity: 0.39750004 time: 0.0040s
Epoch: 0901 Modularity: 0.39750004 time: 0.0030s
Optimization Finished!
Total time elapsed: 2.7939s
Epoch: 0001 Modularity: 0.25809246 time: 0.0010s
Epoch: 0101 Modularity: 0.38941196 time: 0.0010s
Epoch: 0201 Modularity: 0.39750004 time: 0.0020s
Epoch: 0301 Modularity: 0.39750004 time: 0.0020s
Epoch: 0401 Modularity: 0.39750004 time: 0.0010s
Epoch: 0501 Modularity: 0.39750004 time: 0.0020s
Epoch: 0601 Modularity: 0.39750004 time: 0.0020s
Epoch: 0701 Modularity: 0.39750004 time: 0.0020s
Epoch: 0801 Modularity: 0.39750004 time: 0.0020s
Epoch: 0901 Modularity: 0.39750004 time: 0.0010s
Optimization Finis

Epoch: 0401 Modularity: 0.21293184 time: 0.0020s
Epoch: 0501 Modularity: 0.21866006 time: 0.0010s
Epoch: 0601 Modularity: 0.22078627 time: 0.0020s
Epoch: 0701 Modularity: 0.22193506 time: 0.0010s
Epoch: 0801 Modularity: 0.22437158 time: 0.0010s
Epoch: 0901 Modularity: 0.24251574 time: 0.0010s
Optimization Finished!
Total time elapsed: 1.2108s
Epoch: 0001 Modularity: 0.29674834 time: 0.0030s
Epoch: 0101 Modularity: 0.29749745 time: 0.0020s
Epoch: 0201 Modularity: 0.29849780 time: 0.0020s
Epoch: 0301 Modularity: 0.29959273 time: 0.0020s
Epoch: 0401 Modularity: 0.29959273 time: 0.0030s
Epoch: 0501 Modularity: 0.29959273 time: 0.0020s
Epoch: 0601 Modularity: 0.29959273 time: 0.0020s
Epoch: 0701 Modularity: 0.29959273 time: 0.0030s
Epoch: 0801 Modularity: 0.29959273 time: 0.0030s
Epoch: 0901 Modularity: 0.29959273 time: 0.0030s
Optimization Finished!
Total time elapsed: 2.2623s
Epoch: 0001 Modularity: -0.17124999 time: 0.0010s
Epoch: 0101 Modularity: -0.04291525 time: 0.0010s
Epoch: 0201 Mo

In [14]:
dfs

,Network,Method,Communities,Hidden Dimensionality,Initial Modularity,1 Layer Finetuned Moduclarity,2 Layer Finetuned Moduclarity,Combo Modularity
0,Karate Club,louvain,4,5,0.263119,0.419790,0.419790,0.419790
1,Karate Club,spectral,4,5,0.340154,0.399080,0.399080,0.419790
2,Les Miserables,louvain,6,7,0.266517,0.566298,0.566688,0.566688
3,Les Miserables,spectral,6,7,0.117456,0.393088,0.477358,0.566688
4,Florentine,louvain,4,5,0.204602,0.398750,0.398750,0.398750
5,Florentine,spectral,3,4,-0.171250,0.296748,0.299889,0.398750
0,Karate Club,louvain,4,6,0.263119,0.419790,0.419790,0.419790
1,Karate Club,spectral,4,6,0.340154,0.399080,0.399080,0.419790
2,Les Miserables,louvain,6,8,0.266517,0.566298,0.566688,0.566688
3,Les Miserables,spectral,6,8,0.117456,0.393088,0.478821,0.566688
